# TP6 - corps binaires et applications

## Partie 1 - un anneau qui a du caractère

On va travailler dans un **anneau** (structure algébrique où $+, -, \times$ sont autorisées) **fini** à $64$ éléments noté $(\mathbb{F}_{64}, +, \times)$. Les éléments de $\mathbb{F}_{64}$ seront informatiquement des instances de la classe ```F``` implémentée ci-dessous. 

In [135]:
class F(SageObject):
    
    # names for the 64 elements
    # static variables 
    names = "01abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ !?-'#$%&@"
    
    # create F object associated to character val 
    def __init__(self, val):
        
        try:
            # id = integer to identify the ring element 
            self.id = Integer(F.names.index(str(val)))
            
        except ValueError:
            # val is not found in names 
            raise ValueError("Element %s unknown" % val)
            
    # F1 == F2 if F1 and F2 have the same id 
    def __eq__(self, other):
        
        return self.id == other.id
        
    def __ne__(self, other):
        
        return not (self == other)
       
    # special method to represent an instance of F 
    def __repr__(self):
        
        return F.names[self.id]
    
    # resultat of F1 + F2
    def __add__(self, other : 'F'):
        
        r = F(self)
        r += other
        return r
    
    # result of F1 += F2
    def __iadd__(self, other):
        
        # add two objects = bitwise addition on their id 
        self.id = self.id.__xor__(other.id)
        return self
    
    # method that is used to perform F1 * F2 
    def shift(self):
        
        # append a 0 in the binary representation of self.id 
        # tmp is an integer coded with 7 bits 
        tmp = self.id << 1
        
        # res stores bits associated to 2^6 to 2^0 
        res = tmp % 64
        
        # tmp // 64 stores the bit associated to 2^7  
        # if the bit is 1...
        if tmp // 64:
            
            # 1 000 000 is represented by 0 011 011 (why?)
            # perform bitwise addition 011 011 + res  
            res = res.__xor__(27) 
            
        self.id = res
    
    # result of F1 * F2 
    # magic multiplication 
    def __mul__(self, other):
        
        res = F(0)
        tmp = F(other)
        
        # F1 is represented as a binary word 
        # when a bit is 0, do nothing 
        # when a bit is 1, add a shifted version of F(other)
        for b in self.id.bits():
            if b:
                res += tmp
            tmp.shift()
            
        return res
        
    # resultat of F*F*...*F
    def __pow__(self, exp):
        
        res = F(1)
        
        for i in range(exp):
            res = res * self
        
        return res
        
F.elements = [F(i) for i in F.names]

### a) C'est un anneau commutatif

Vérifier explicitement (par *force brute*) que **tous** les axiomes d'anneau commutatif sont satisfaits pour cette structure algébrique.

In [136]:
alpha = choice(F.elements)  # random element, run again for another one
beta = choice(F.elements)  # random element, run again for another one
gamma = choice(F.elements)  # random element, run again for another one

In [137]:
# Test lci
print(f"F est une lci pour \"+\" : {(alpha + beta) in F.elements}")
print(f"F est une lci pour \"*\" : {(alpha * beta) in F.elements}")

F est une lci pour "+" : True
F est une lci pour "*" : True


In [138]:
# Test associativité
print(f"F est associatif pour \"+\" : {(alpha + beta) + gamma == alpha + (beta + gamma)}")
print(f"F est associatif pour \"*\" : {(alpha * beta) * gamma == alpha * (beta * gamma)}")

F est associatif pour "+" : True
F est associatif pour "*" : True


In [139]:
# Test commutativité
print(f"F est commutatif pour \"+\" : {alpha + beta == beta + alpha}")
print(f"F est commutatif pour \"*\" : {alpha * beta == beta * alpha}")

F est commutatif pour "+" : True
F est commutatif pour "*" : True


In [140]:
# Test neutre
print(f"F a un élément neutre pour \"+\" : {(alpha + F(0)) == alpha}")
print(f"F a un élément neutre pour \"*\" : {(alpha * F(1)) == alpha}")

F a un élément neutre pour "+" : True
F a un élément neutre pour "*" : True


In [141]:
# Test opposé
print(f"F a un élément opposé pour \"+\" : {(alpha + alpha) == F(0)}")

F a un élément opposé pour "+" : True


In [142]:
# Test "*" est distributif par rapport à "+"
print(f"F est distributif pour \"*\" : {(alpha * (beta + gamma)) == (alpha * beta) + (alpha * gamma)}")

F est distributif pour "*" : True


### b) Inverse d'un élément

Soit $m$ la somme des numéros des mois de naissance des membres du binôme et $\zeta$ l'élément `F.elements[m]` de $\mathbb{F}_{64}$. Vérifiez que $\zeta$ est inversible en recherchant par force brute son inverse parmi les éléments de $\mathbb{F}_{64}$.

In [143]:
# Trouver l'inverse de zeta
zeta = F.elements[10 + 2] # Octobre + Fevrier
print(f"zeta = {zeta}")

# On cherche l'inverse de zeta en testant tous les éléments de F si zeta * x = 1
izeta = [x for x in F.elements if x * zeta == F(1)][0]

print(f"L'inverse de \"{zeta}\" est \"{izeta}\" : {zeta} * {izeta} = {zeta * F(izeta)}")

zeta = k
L'inverse de "k" est "E" : k * E = 1


### c) Structure multiplicative

- Déterminer l'ordre multiplicatif de l'élément $\mathtt{a}$

In [144]:
# Determiner l'ordre multiplicatif de alpha

alpha = F("a")
count = 1
beta = alpha
while beta != F(1):
    beta = alpha ** count
    print(f"{alpha}^{count} = {beta}")
    count += 1
count -= 1 # count is incremented one time too much
print(f"L'ordre multiplicatif de \"{alpha}\" est {count}")

a^1 = a
a^2 = c
a^3 = g
a^4 = o
a^5 = E
a^6 = z
a^7 =  
a^8 = !
a^9 = Z
a^10 = V
a^11 = -
a^12 = N
a^13 = h
a^14 = q
a^15 = I
a^16 = r
a^17 = K
a^18 = v
a^19 = S
a^20 = f
a^21 = m
a^22 = A
a^23 = ?
a^24 = P
a^25 = l
a^26 = y
a^27 = Y
a^28 = X
a^29 = %
a^30 = F
a^31 = x
a^32 = W
a^33 = @
a^34 = J
a^35 = p
a^36 = G
a^37 = D
a^38 = &
a^39 = L
a^40 = t
a^41 = O
a^42 = n
a^43 = C
a^44 = $
a^45 = H
a^46 = B
a^47 = '
a^48 = T
a^49 = d
a^50 = i
a^51 = s
a^52 = M
a^53 = j
a^54 = u
a^55 = Q
a^56 = b
a^57 = e
a^58 = k
a^59 = w
a^60 = U
a^61 = #
a^62 = R
a^63 = 1
L'ordre multiplicatif de "a" est 63


- En déduire que tous les éléments de $\mathbb{F}_{64}$ (sauf le neutre additif) sont inversibles. 

Notre ordre multiplicatif est donc 63, ce qui signifie qu'on a 63 élements inversible donc que tous les éléments de $\mathbb{F}_{64}$ sont inversibles sauf le neutre additif.


- En partant de $\mathtt{a}^{-1} = \mathtt{R}$, déterminer astucieusement (donc pas par force brute) l'inverse de $\zeta$

On remarque que $\mathtt{a}^{-1} = \mathtt{R} = \mathtt{a}^{62}$ donc $\mathtt{\zeta}^{-1} = \mathtt{\zeta}^{62}$

In [145]:
izeta = zeta ** 62
print(f"L'inverse de \"{zeta}\" est \"{izeta}\" : {zeta} * {izeta} = {zeta * F(izeta)}")

L'inverse de "k" est "E" : k * E = 1


### d) Solutions d'une équation polynomiale

- Déterminer les racines dans $\mathbb{F}_{64}$ du polynôme $X^6 + X^4 + X^3 + X + 1 \in \mathbb{F}_2[X]$

In [146]:
def poly(x):
    return x**6 + x**4 + x**3 + x + F(1)

root = [x for x in F.elements if poly(x) == F(0)]

print(f"Les racines du polynome sont {root}")

Les racines du polynome sont [a, c, o, r, W, !]


- Expliquer comment cela permet de se convaincre que chaque élément de $\mathbb{F}_{64}$ peut s'écrire, de façon unique, sous la forme d'une expression de la forme

    $$ \boxed{b_5 \mathtt{a}^5 + b_4 \mathtt{a}^4 + b_3 \mathtt{a}^3 + b_2 \mathtt{a}^2 + b_1 \mathtt{a} + b_0} $$

    avec $(b_0, \ldots, b_5) \in \mathbb{F}_2^6$

Comme on sait que $x + x = 0$ avec $x \in \mathbb{F}_{64}$, ça implique que $a^6 = a^4 + a^3 + a + 1$, donc ici $b_5 = 0$, $b_4 = 1$, $b_3 = 1$, $b_2 = 0$, $b_1 = 1$, $b_0 = 1$. 

C'est comme si on l'écrivait sous la forme d'un nombre binaire à 6 chiffres. $a^6 = z = 011011$


In [147]:
def unique(b5, b4, b3, b2, b1, b0):
    return F(b0) + F(b1) * F('a') + F(b2) * F('a')**2 + F(b3) * F('a')**3 + F(b4) * F('a')**4 + F(b5) * F('a')**5 

In [148]:
unique(0, 1, 1, 0, 1, 1)

z

- Quelle est l'écriture correspondante pour votre élément $\zeta$ ci-dessus ?

In [149]:
print(f"zeta = {zeta}")
count = 1
while F('a') ** count != F('k'):
    count += 1
count

zeta = k


58

In [150]:
# choose every combinasion of six elements of [0,1] then test if unique(them) == k
for b5 in [0,1]:
    for b4 in [0,1]:
        for b3 in [0,1]:
            for b2 in [0,1]:
                for b1 in [0,1]:
                    for b0 in [0,1]:
                        if unique(b5, b4, b3, b2, b1, b0) == F('k'):
                            print(f"unique({b5}, {b4}, {b3}, {b2}, {b1}, {b0}) = {F('k')}")

unique(0, 0, 1, 1, 0, 0) = k


Pour notre élément $\zeta$, on a $b_5 = 0$, $b_4 = 0$, $b_3 = 1$, $b_2 = 1$, $b_1 = 0$, $b_0 = 0$.  
Ou $\zeta = a^3 + a^2$

## Partie 2 - chaînes de caractères et applications

Les éléments de $\mathbb{F}_{64}$ étant des caractères, on considérer une chaîne de caractères de longueur $n$ comme un élément de $\mathbb{F}_{64}^n$. Voici un début de classe permettant de considérer une chaîne de caractères (admissibles) comme une liste d'élémentsde $\mathbb{F}_{64}$.

In [151]:
class V(SageObject):
    
    # constructor 
    # s is a string of eligible characters 
    def __init__(self, s):
        
        self.coeffs = [ F(c) for c in s ]
        
    # representation of a V object 
    def __repr__(self):
        
        return "".join([c.__repr__() for c in self.coeffs])
    
    # result of V1 == V2 
    def __eq__(self, other):
        
        return self.coeffs == other.coeffs
   
    # length of s
    def __len__(self):
        
        return len(self.coeffs)
    
    # somme terme à terme
    def __add__(self, other):
        if len(self) != len(other):
            raise ValueError("Les mots de code ne sont pas de même longueur")
        res = V("")
        for i in range(len(self)):
            res.coeffs.append(self.coeffs[i] + other.coeffs[i])
        return res

    
    def controle(self):
        """Donne le charactère de contrôle d'un mot de code"""
        c = F(0)
        for i in self.coeffs:
            c += i
        return c  

### a) Somme de vecteurs

- Ajouter à la classe `V` une méthode `__add__` permettant de faire la somme **terme à terme** de deux chaînes de caractères de même longueur.

J'ai ajouté la méthode `__add__` à la classe `V` qui permet de faire la somme terme à terme de deux chaînes de caractères de même longueur.
```python
    def __add__(self, other):
        if len(self) != len(other):
            raise ValueError("Les mots de code ne sont pas de même longueur")
        res = V("")
        for i in range(len(self)):
            res.coeffs.append(self.coeffs[i] + other.coeffs[i])
        return res
```

In [152]:
print(f"V(\"ceci\") + V(\"cela\") == V(\"00hg\") : {V('ceci') + V('cela') == V('00hg')}")
print(f"V(\"CECI\") + V(\"cela\") == V(\"yKrK\") : {V('CECI') + V('cela') == V('yKrK')}")
print(f"V(\"ceci\") + V(\"CELA\") == V(\"yKHu\") : {V('ceci') + V('CELA') == V('yKHu')}")
print(f"V(\"CECI\") + V(\"CELA\") == V(\"00-?\") : {V('CECI') + V('CELA') == V('00-?')}")


V("ceci") + V("cela") == V("00hg") : True
V("CECI") + V("cela") == V("yKrK") : True
V("ceci") + V("CELA") == V("yKHu") : True
V("CECI") + V("CELA") == V("00-?") : True


### b) Chiffrement à clé secrète

On peut utiliser l'addition de chaînes de caractères ainsi définie pour garantir la confidentialité d'une chaîne de caractères donnée. Par exemple : soit le message clair `m` ci-dessous. 

In [153]:
m = V("Texte clair")

On peut le rendre inintelligible par un tiers en le masquant à l'aide d'une clé secrète `k` générée aléatoirement: 

In [154]:
k = V("kqaS%LAKVw'")

On obtient alors la chaîne incompréhensible suivante calculant la *somme* `c` de `m` et `k`: 

In [155]:
c = V("Hsz##pwPXqN")

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

En utilisant les propriétés de $\mathbb{F}_{64}$: 
- indiquer comment déchiffrer le code si la clé est connu 

Pour coder on fait m + k, pour décoder on fait c - k mais comme on est dans $\mathbb{F}_{64}$, on sais que + = - donc on fait c + k

In [156]:
print(f"Pour m = {m}, k = {k}, c = {c}, c = m + k : {c == m + k}")
print(f"Pour m = {m}, k = {k}, c = {c}, m = c + k : {m == c + k}")

Pour m = Texte clair, k = kqaS%LAKVw', c = Hsz##pwPXqN, c = m + k : True
Pour m = Texte clair, k = kqaS%LAKVw', c = Hsz##pwPXqN, m = c + k : True


- déchiffrer le message `V("Kcgw&YCVFSl")` codé avec la même clé 

In [157]:
m_code = V("Kcgw&YCVFSl")
print(f"Le message décodé est : {m_code + k}")

Le message décodé est : Oui bravo !


### c) Détection d'erreurs

- Implémenter une fonction pour calculer la somme de contrôle d'un élément de $\mathbb{F}_{64}^n$. Déterminer la somme de contrôle de la phrase `It's burger time!` 

J'ai ici implémenté une fonction qui calcule la somme de contrôle d'un élément de $\mathbb{F}_{64}^n$. Elle fait la somme de tous les caractères de la phrase.
```python
    def controle(self):
        """Donne le charactère de contrôle d'un mot de code"""
        c = F(0)
        for i in self.coeffs:
            c += i
        return c  
```

In [158]:
message = V("It's burger time !")
message.controle()

q

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

On peut détecter des erreurs si la somme de contrôle calculée ne correspond pas à la somme de contrôle attendue. Comprenez-le avec l'exemple ci-dessous. 

In [159]:
v = V("Ici y a-t-il une erreur ? C#mment le savez-vous ?")
s = F("Z") # expected checksum 

In [160]:
v_controle = v.controle()

print(f"La valeur de contrôle de v est {v_controle} : {v_controle == s}")

La valeur de contrôle de v est C : False


On voit donc que notre controle ne valide pas le checksum, ça signifie qu'il y a une erreur dans le message.

In [161]:
v_good = V("Ici y a-t-il une erreur ? Comment le savez-vous ?")
v_good_controle = v_good.controle()

print(f"La valeur de contrôle de v_good est {v_good_controle} : {v_good_controle == s}")

La valeur de contrôle de v_good est Z : True


Avec cette fois un message sans erreur, on voit que le checksum est valide.